<a href="https://colab.research.google.com/github/erkamalkumar/Pytorch-from-scratch/blob/main/Gradient_descent_and_linear_regression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import numpy as np
import torch

In [3]:
# Input (temp, rainfall, humidity)
inputs = np.array([[74, 68, 44], 
                   [92, 89, 65], 
                   [88, 135, 59], 
                   [103, 44, 38], 
                   [70, 97, 71]], dtype='float32')

# Targets (bananas, pineapples)
targets = np.array([[57, 71], 
                    [82, 102], 
                    [120, 134], 
                    [23, 38], 
                    [104, 120]], dtype='float32')

In [4]:
# Convert inputs and targets to tensors
inputs = torch.from_numpy(inputs)
targets = torch.from_numpy(targets)
print(inputs)
print(targets)

tensor([[ 74.,  68.,  44.],
        [ 92.,  89.,  65.],
        [ 88., 135.,  59.],
        [103.,  44.,  38.],
        [ 70.,  97.,  71.]])
tensor([[ 57.,  71.],
        [ 82., 102.],
        [120., 134.],
        [ 23.,  38.],
        [104., 120.]])


## Linear regression model from scratch

The weights and biases (`w11, w12,... w23, b1 & b2`) can also be represented as matrices, initialized as random values. The first row of `w` and the first element of `b` are used to predict the first target variable, i.e., yield of bananas, and similarly, the second for pineapples.

In [5]:
# Weights and biases
w = torch.randn(2, 3, requires_grad=True)
b = torch.randn(2, requires_grad=True)
print(w)
print(b)

tensor([[ 2.8233,  1.3696,  0.6327],
        [ 1.4846, -2.7422, -0.0639]], requires_grad=True)
tensor([ 1.6710, -0.2530], requires_grad=True)


`torch.randn` creates a tensor with the given shape, with elements picked randomly from a [normal distribution](https://en.wikipedia.org/wiki/Normal_distribution) with mean 0 and standard deviation 1.

Our *model* is simply a function that performs a matrix multiplication of the `inputs` and the weights `w` (transposed) and adds the bias `b` (replicated for each observation).

![matrix-mult](https://i.imgur.com/WGXLFvA.png)

We can define the model as follows:

In [6]:
def model(x):
    return x @ w.t() + b

`@` represents matrix multiplication in PyTorch, and the `.t` method returns the transpose of a tensor.

The matrix obtained by passing the input data into the model is a set of predictions for the target variables.

In [7]:
# Generate predictions
preds = model(inputs)
print(preds)

tensor([[ 331.5681,  -79.6739],
        [ 424.4366, -111.8796],
        [ 472.3499, -243.5766],
        [ 376.7755,   29.5774],
        [ 377.0780, -166.8629]], grad_fn=<AddBackward0>)


Let's compare the predictions of our model with the actual targets.

In [8]:
# Compare with targets
print(targets)

tensor([[ 57.,  71.],
        [ 82., 102.],
        [120., 134.],
        [ 23.,  38.],
        [104., 120.]])


You can see a big difference between our model's predictions and the actual targets because we've initialized our model with random weights and biases. Obviously, we can't expect a randomly initialized model to *just work*.

## Loss function

Before we improve our model, we need a way to evaluate how well our model is performing. We can compare the model's predictions with the actual targets using the following method:

* Calculate the difference between the two matrices (`preds` and `targets`).
* Square all elements of the difference matrix to remove negative values.
* Calculate the average of the elements in the resulting matrix.

The result is a single number, known as the **mean squared error** (MSE).

In [9]:
# MSE loss
def mse(t1, t2):
    diff = t1 - t2
    return torch.sum(diff * diff) / diff.numel()

`torch.sum` returns the sum of all the elements in a tensor. The `.numel` method of a tensor returns the number of elements in a tensor. Let's compute the mean squared error for the current predictions of our model.

In [10]:
# Compute loss
loss = mse(preds, targets)
print(loss)

tensor(80990.2188, grad_fn=<DivBackward0>)


## Compute gradients

With PyTorch, we can automatically compute the gradient or derivative of the loss w.r.t. to the weights and biases because they have `requires_grad` set to `True`. We'll see how this is useful in just a moment.

In [11]:
# Compute gradients
loss.backward()

The gradients are stored in the `.grad` property of the respective tensors. Note that the derivative of the loss w.r.t. the weights matrix is itself a matrix with the same dimensions.

In [12]:
# Gradients for weights
print(w)
print(w.grad)

tensor([[ 2.8233,  1.3696,  0.6327],
        [ 1.4846, -2.7422, -0.0639]], requires_grad=True)
tensor([[ 27676.6699,  27753.8867,  17592.0078],
        [-17000.2930, -21690.0508, -12699.2344]])


In [13]:
w
w.grad

tensor([[ 27676.6699,  27753.8867,  17592.0078],
        [-17000.2930, -21690.0508, -12699.2344]])

In [14]:
with torch.no_grad():
    w -= w.grad * 1e-5
    b -= b.grad * 1e-5

We multiply the gradients with a very small number (`10^-5` in this case) to ensure that we don't modify the weights by a very large amount. We want to take a small step in the downhill direction of the gradient, not a giant leap. This number is called the *learning rate* of the algorithm. 

We use `torch.no_grad` to indicate to PyTorch that we shouldn't track, calculate, or modify gradients while updating the weights and biases.

In [15]:
# Let's verify that the loss is actually lower
loss = mse(preds, targets)
print(loss)

tensor(80990.2188, grad_fn=<DivBackward0>)


Before we proceed, we reset the gradients to zero by invoking the `.zero_()` method. We need to do this because PyTorch accumulates gradients. Otherwise, the next time we invoke `.backward` on the loss, the new gradient values are added to the existing gradients, which may lead to unexpected results.

In [16]:
w.grad.zero_()
b.grad.zero_()
print(w.grad)
print(b.grad)

tensor([[0., 0., 0.],
        [0., 0., 0.]])
tensor([0., 0.])


## Train the model using gradient descent

As seen above, we reduce the loss and improve our model using the gradient descent optimization algorithm. Thus, we can _train_ the model using the following steps:

1. Generate predictions

2. Calculate the loss

3. Compute gradients w.r.t the weights and biases

4. Adjust the weights by subtracting a small quantity proportional to the gradient

5. Reset the gradients to zero

Let's implement the above step by step.

In [17]:
# Generate predictions
preds = model(inputs)
print(preds)

tensor([[ 284.4710,  -46.7547],
        [ 362.8351,  -68.6786],
        [ 400.1442, -191.8401],
        [ 329.3687,   61.4591],
        [ 318.2896, -124.9048]], grad_fn=<AddBackward0>)


In [18]:
# Calculate the loss
loss = mse(preds, targets)
print(loss)

tensor(55857.1797, grad_fn=<DivBackward0>)


In [19]:
# Compute gradients
loss.backward()
print(w.grad)
print(b.grad)

tensor([[ 22775.7266,  22509.6270,  14329.6172],
        [-13563.4531, -17981.9414, -10399.3369]])
tensor([ 261.8217, -167.1438])


Let's update the weights and biases using the gradients computed above.

In [20]:
# Adjust weights & reset gradients
with torch.no_grad():
    w -= w.grad * 1e-5
    b -= b.grad * 1e-5
    w.grad.zero_()
    b.grad.zero_()

Let's take a look at the new weights and biases.

In [21]:
print(w)
print(b)

tensor([[ 2.3187,  0.8670,  0.3135],
        [ 1.7903, -2.3455,  0.1671]], requires_grad=True)
tensor([ 1.6652, -0.2493], requires_grad=True)


With the new weights and biases, the model should have a lower loss.

In [22]:
# Calculate loss
preds = model(inputs)
loss = mse(preds, targets)
print(loss)

tensor(39082.5859, grad_fn=<DivBackward0>)


We have already achieved a significant reduction in the loss merely by adjusting the weights and biases slightly using gradient descent.

## Train for multiple epochs

To reduce the loss further, we can repeat the process of adjusting the weights and biases using the gradients multiple times. Each iteration is called an _epoch_. Let's train the model for 100 epochs.

In [23]:
# Train for 100 epochs
for i in range(100):
    preds = model(inputs)
    loss = mse(preds, targets)
    loss.backward()
    with torch.no_grad():
        w -= w.grad * 1e-5
        b -= b.grad * 1e-5
        w.grad.zero_()
        b.grad.zero_()

Once again, let's verify that the loss is now lower:

In [24]:
# Calculate loss
preds = model(inputs)
loss = mse(preds, targets)
print(loss)

tensor(1623.1674, grad_fn=<DivBackward0>)


The loss is now much lower than its initial value. Let's look at the model's predictions and compare them with the targets.

In [25]:
# Predictions
preds

tensor([[ 67.7175,  86.6552],
        [ 83.8605, 113.4102],
        [102.6350,  82.1479],
        [ 75.7292, 127.5108],
        [ 73.0631,  88.4049]], grad_fn=<AddBackward0>)

In [26]:
# Targets
targets

tensor([[ 57.,  71.],
        [ 82., 102.],
        [120., 134.],
        [ 23.,  38.],
        [104., 120.]])

The predictions are now quite close to the target variables. We can get even better results by training for a few more epochs. 

## Linear regression using PyTorch built-ins

We've implemented linear regression & gradient descent model using some basic tensor operations. However, since this is a common pattern in deep learning, PyTorch provides several built-in functions and classes to make it easy to create and train models with just a few lines of code.

Let's begin by importing the `torch.nn` package from PyTorch, which contains utility classes for building neural networks.

In [27]:
import torch.nn as nn

As before, we represent the inputs and targets and matrices.

In [28]:
# Input (temp, rainfall, humidity)
inputs = np.array([[74, 68, 44], 
                   [92, 89, 65], 
                   [88, 135, 59], 
                   [103, 44, 38], 
                   [70, 97, 71], 
                   [75, 67, 44], 
                   [92, 88, 66], 
                   [89, 135, 60], 
                   [102, 45, 38], 
                   [69, 97, 72], 
                   [74, 67, 45], 
                   [93, 88, 65], 
                   [88, 136, 58], 
                   [104, 44, 37], 
                   [69, 98, 71]], 
                  dtype='float32')

# Targets (bananas, pineapples)
targets = np.array([[57, 71], 
                    [82, 102], 
                    [120, 134], 
                    [23, 38], 
                    [104, 120],
                    [58, 60], 
                    [81, 103], 
                    [119, 133], 
                    [22, 39], 
                    [105, 119], 
                    [58, 70], 
                    [83, 101], 
                    [119, 135], 
                    [21, 39], 
                    [103, 121]], 
                   dtype='float32')

inputs = torch.from_numpy(inputs)
targets = torch.from_numpy(targets)

In [29]:
inputs

tensor([[ 74.,  68.,  44.],
        [ 92.,  89.,  65.],
        [ 88., 135.,  59.],
        [103.,  44.,  38.],
        [ 70.,  97.,  71.],
        [ 75.,  67.,  44.],
        [ 92.,  88.,  66.],
        [ 89., 135.,  60.],
        [102.,  45.,  38.],
        [ 69.,  97.,  72.],
        [ 74.,  67.,  45.],
        [ 93.,  88.,  65.],
        [ 88., 136.,  58.],
        [104.,  44.,  37.],
        [ 69.,  98.,  71.]])

We are using 15 training examples to illustrate how to work with large datasets in small batches. 

## Dataset and DataLoader

We'll create a `TensorDataset`, which allows access to rows from `inputs` and `targets` as tuples, and provides standard APIs for working with many different types of datasets in PyTorch.

In [30]:
from torch.utils.data import TensorDataset

In [31]:
# Define dataset
train_ds = TensorDataset(inputs, targets)
train_ds[0:3]

(tensor([[ 74.,  68.,  44.],
         [ 92.,  89.,  65.],
         [ 88., 135.,  59.]]),
 tensor([[ 57.,  71.],
         [ 82., 102.],
         [120., 134.]]))

The `TensorDataset` allows us to access a small section of the training data using the array indexing notation (`[0:3]` in the above code). It returns a tuple with two elements. The first element contains the input variables for the selected rows, and the second contains the targets.

We'll also create a `DataLoader`, which can split the data into batches of a predefined size while training. It also provides other utilities like shuffling and random sampling of the data.

In [32]:
from torch.utils.data import DataLoader

In [33]:
# Define data loader
batch_size = 5
train_dl = DataLoader(train_ds, batch_size, shuffle=True)

We can use the data loader in a `for` loop. Let's look at an example.

In [34]:
for xb, yb in train_dl:
    print(xb)
    print(yb)
    break

tensor([[ 70.,  97.,  71.],
        [102.,  45.,  38.],
        [ 92.,  89.,  65.],
        [ 69.,  97.,  72.],
        [ 74.,  67.,  45.]])
tensor([[104., 120.],
        [ 22.,  39.],
        [ 82., 102.],
        [105., 119.],
        [ 58.,  70.]])


In each iteration, the data loader returns one batch of data with the given batch size. If `shuffle` is set to `True`, it shuffles the training data before creating batches. Shuffling helps randomize the input to the optimization algorithm, leading to a faster reduction in the loss.

## nn.Linear

Instead of initializing the weights & biases manually, we can define the model using the `nn.Linear` class from PyTorch, which does it automatically.

In [35]:
# Define model
model = nn.Linear(3, 2)
print(model.weight)
print(model.bias)

Parameter containing:
tensor([[-0.0271, -0.0208,  0.2332],
        [-0.0851, -0.5417,  0.1748]], requires_grad=True)
Parameter containing:
tensor([-0.3751,  0.2542], requires_grad=True)


PyTorch models also have a helpful `.parameters` method, which returns a list containing all the weights and bias matrices present in the model. For our linear regression model, we have one weight matrix and one bias matrix.

In [36]:
# Parameters
list(model.parameters())

[Parameter containing:
 tensor([[-0.0271, -0.0208,  0.2332],
         [-0.0851, -0.5417,  0.1748]], requires_grad=True),
 Parameter containing:
 tensor([-0.3751,  0.2542], requires_grad=True)]

We can use the model to generate predictions in the same way as before.

In [37]:
# Generate predictions
preds = model(inputs)
preds

tensor([[  6.4647, -35.1897],
        [ 10.4371, -44.4272],
        [  8.1892, -70.0557],
        [  4.7785, -25.7040],
        [ 12.2664, -45.8406],
        [  6.4584, -34.7330],
        [ 10.6911, -43.7107],
        [  8.3953, -69.9660],
        [  4.7848, -26.1607],
        [ 12.5267, -45.5807],
        [  6.7187, -34.4732],
        [ 10.4308, -43.9706],
        [  7.9352, -70.7722],
        [  4.5181, -25.9639],
        [ 12.2727, -46.2973]], grad_fn=<AddmmBackward0>)

## Loss Function

Instead of defining a loss function manually, we can use the built-in loss function `mse_loss`.

In [38]:
# Import nn.functional
import torch.nn.functional as F

The `nn.functional` package contains many useful loss functions and several other utilities. 

In [39]:
# Define loss function
loss_fn = F.mse_loss

Let's compute the loss for the current predictions of our model.

In [40]:
loss = loss_fn(model(inputs), targets)
print(loss)

tensor(13392.1602, grad_fn=<MseLossBackward0>)


## Optimizer

Instead of manually manipulating the model's weights & biases using gradients, we can use the optimizer `optim.SGD`. SGD is short for "stochastic gradient descent". The term _stochastic_ indicates that samples are selected in random batches instead of as a single group.

In [41]:
# Define optimizer
opt = torch.optim.SGD(model.parameters(), lr=1e-5)

Note that `model.parameters()` is passed as an argument to `optim.SGD` so that the optimizer knows which matrices should be modified during the update step. Also, we can specify a learning rate that controls the amount by which the parameters are modified.

## Train the model

We are now ready to train the model. We'll follow the same process to implement gradient descent:

1. Generate predictions

2. Calculate the loss

3. Compute gradients w.r.t the weights and biases

4. Adjust the weights by subtracting a small quantity proportional to the gradient

5. Reset the gradients to zero

The only change is that we'll work batches of data instead of processing the entire training data in every iteration. Let's define a utility function `fit` that trains the model for a given number of epochs.

In [42]:
# Utility function to train the model
def fit(num_epochs, model, loss_fn, opt, train_dl):
    
    # Repeat for given number of epochs
    for epoch in range(num_epochs):
        
        # Train with batches of data
        for xb,yb in train_dl:
            
            # 1. Generate predictions
            pred = model(xb)
            
            # 2. Calculate loss
            loss = loss_fn(pred, yb)
            
            # 3. Compute gradients
            loss.backward()
            
            # 4. Update parameters using gradients
            opt.step()
            
            # 5. Reset the gradients to zero
            opt.zero_grad()
        
        # Print the progress
        if (epoch+1) % 10 == 0:
            print('Epoch [{}/{}], Loss: {:.4f}'.format(epoch+1, num_epochs, loss.item()))

Some things to note above:

* We use the data loader defined earlier to get batches of data for every iteration.

* Instead of updating parameters (weights and biases) manually, we use `opt.step` to perform the update and `opt.zero_grad` to reset the gradients to zero.

* We've also added a log statement that prints the loss from the last batch of data for every 10th epoch to track training progress. `loss.item` returns the actual value stored in the loss tensor.

Let's train the model for 100 epochs.

In [43]:
fit(100, model, loss_fn, opt, train_dl)

Epoch [10/100], Loss: 460.5307
Epoch [20/100], Loss: 290.8167
Epoch [30/100], Loss: 195.8732
Epoch [40/100], Loss: 97.5310
Epoch [50/100], Loss: 144.2615
Epoch [60/100], Loss: 40.3986
Epoch [70/100], Loss: 65.4215
Epoch [80/100], Loss: 23.4482
Epoch [90/100], Loss: 21.7577
Epoch [100/100], Loss: 22.4213


Let's generate predictions using our model and verify that they're close to our targets.

In [44]:
# Generate predictions
preds = model(inputs)
preds

tensor([[ 59.0172,  72.3908],
        [ 82.8583, 100.8345],
        [117.9581, 131.5566],
        [ 28.1313,  45.6078],
        [ 98.8120, 114.5554],
        [ 57.9282,  71.4896],
        [ 82.6068, 100.8175],
        [118.2418, 132.1391],
        [ 29.2204,  46.5090],
        [ 99.6496, 115.4395],
        [ 58.7658,  72.3737],
        [ 81.7692,  99.9333],
        [118.2096, 131.5737],
        [ 27.2938,  44.7237],
        [ 99.9011, 115.4566]], grad_fn=<AddmmBackward0>)

In [45]:
# Compare with targets
targets

tensor([[ 57.,  71.],
        [ 82., 102.],
        [120., 134.],
        [ 23.,  38.],
        [104., 120.],
        [ 58.,  60.],
        [ 81., 103.],
        [119., 133.],
        [ 22.,  39.],
        [105., 119.],
        [ 58.,  70.],
        [ 83., 101.],
        [119., 135.],
        [ 21.,  39.],
        [103., 121.]])

Indeed, the predictions are quite close to our targets. We have a trained a reasonably good model to predict crop yields for bananas and pineapples by looking at the average temperature, rainfall, and humidity in a region. We can use it to make predictions of crop yields for new regions by passing a batch containing a single row of input.

In [46]:
model(torch.tensor([[75, 63, 44.]]))

tensor([[54.6799, 68.4880]], grad_fn=<AddmmBackward0>)